In [1]:
from sklearn.linear_model import LogisticRegression
import importlib
import ClassifierDataPrepper
importlib.reload(ClassifierDataPrepper)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics, preprocessing
import nltk
from nltk.stem import WordNetLemmatizer
import spacy
import pandas as pd

dataPath = "./"
trainingDataPath = dataPath + "train/"

positiveTrainingDataPath = trainingDataPath + "pos/"
negativeTrainingDataPath = trainingDataPath + "neg/"
testDataPath = dataPath + "test/"
# positiveTrainingDataPath = trainingDataPath + "pos_small/"
# negativeTrainingDataPath = trainingDataPath + "neg_small/"
# testDataPath = dataPath + "test_small/"
# testDataPath = None
print("Opening training and test files...")
cdp = ClassifierDataPrepper.ClassifierDataPrepper(positiveTrainingDataPath, negativeTrainingDataPath, testDataPath)

print("Preparing data frames...")
X, Y = cdp.getXYlabeled()
Z = cdp.getXtest()

print("Splitting the data only once and measuring performance for our trained model...")
X_train_pd, X_validate_pd, Y_train, Y_validate = train_test_split(X, Y, test_size=0.2, random_state=101)

print("Splitting comments into words and removing html tags...")
X_train_words = []
X_train = []
for comment in X_train_pd:
    comment = cdp.cleanhtml(comment).lower()
    X_train.append(comment)
    comment = comment.split()
    X_train_words.append(comment)   
print(X_train[0])    
X_validate_words = [] 
X_validate = []
for comment in X_validate_pd:
    comment = cdp.cleanhtml(comment).lower()
    X_validate.append(comment)
    comment = comment.split()
    X_validate_words.append(comment)    
    
Z_words = []
for comment in Z:
    comment = cdp.cleanhtml(comment)
    comment = comment.lower().split()
    Z_words.append(comment)
    
Y_words = []
for label in Y_train:
    Y_words.append(label)    
for label in Y_validate:
    Y_words.append(label)       
    
print(len(X_train_words))
print(len(Y_words))
print(len(Z_words))
print(len(X_train))
print(len(X_validate))
print("done!!!")


Opening training and test files...

Preparing data frames...

Splitting the data only once and measuring performance for our trained model...

Splitting comments into words and removing html tags...

that s what one of the girls said at the end     is the soccer game a metaphor for a qualifying game between the girls    or more broadly    a free thinking group    and the authority       to germany    means to a future that s of hope      it s one of the most unforgettable cinematic experience i ve ever had    despite the crude cinematography and plot    and mild over acting    though i like the cast    they re lovable and well above the expectation for amateurs       the ridiculous situation is well captured    i can feel the deep frustration being denied to a game    being female and a soccer fan    and i cannot stop thinking how to make a convincing disguise    i wonder why there s no women s section in which protection from dirty language and bad behavior can be provided    defeating the flawed reasons for the deny     the movie is very cleverly made    the amazing title    the filming during the actual game    the spontaneity    and various methods to put the viewers into the s

...

25000

25000

20000

5000

done!!!

In [2]:
# Split labelled data into train and validation sets
print("training doc2vec...")
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = []
idx = 0
for comment in X_train_words:
    documents.append(TaggedDocument(comment, [idx]))
    idx += 1
    
# for comment in X_validate_words:
#     documents.append(TaggedDocument(comment, [idx]))
#     idx += 1    

print(documents[0])
print(len(documents))

from random import shuffle
shuffle(documents)

print("Training doc2vec model...")
d2v_model = Doc2Vec(vector_size=400, min_count=2, epochs=30, workers=8)
d2v_model.build_vocab(documents)
d2v_model.train(documents, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
d2v_model.save(fname)

print("done!")

training doc2vec...

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


TaggedDocument(['that', 's', 'what', 'one', 'of', 'the', 'girls', 'said', 'at', 'the', 'end', 'is', 'the', 'soccer', 'game', 'a', 'metaphor', 'for', 'a', 'qualifying', 'game', 'between', 'the', 'girls', 'or', 'more', 'broadly', 'a', 'free', 'thinking', 'group', 'and', 'the', 'authority', 'to', 'germany', 'means', 'to', 'a', 'future', 'that', 's', 'of', 'hope', 'it', 's', 'one', 'of', 'the', 'most', 'unforgettable', 'cinematic', 'experience', 'i', 've', 'ever', 'had', 'despite', 'the', 'crude', 'cinematography', 'and', 'plot', 'and', 'mild', 'over', 'acting', 'though', 'i', 'like', 'the', 'cast', 'they', 're', 'lovable', 'and', 'well', 'above', 'the', 'expectation', 'for', 'amateurs', 'the', 'ridiculous', 'situation', 'is', 'well', 'captured', 'i', 'can', 'feel', 'the', 'deep', 'frustration', 'being', 'denied', 'to', 'a', 'game', 'being', 'female', 'and', 'a', 'soccer', 'fan', 'and', 'i', 'cannot', 'stop', 'thinking', 'how', 'to', 'make', 'a', 'convincing', 'disguise', 'i', 'wonder', 'w

20000

Training doc2vec model...

done!

In [3]:
# print("Loading d2vecmodel from file")
# d2v_model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
# 
# print("done!")

In [4]:
print("Getting vectors from data...")

X_train_d2v = []
for i, comment in enumerate(X_train_words):
    X_train_d2v.append(d2v_model.infer_vector(comment))
    if not i % 2000:
        print("parsed {} comments...".format(i))
    
X_validate_d2v = []
for i, comment in enumerate(X_validate_words):
    X_validate_d2v.append(d2v_model.infer_vector(comment))
    if not i % 2000:
        print("parsed {} comments...".format(i))    
    
# Z_d2v = []
# for comment in Z_words:
#     Z_d2v.append(d2v_model.infer_vector(comment))
    
# print(X_validate_d2v[0])
print("done!")


Getting vectors from data...

parsed 0 comments...

parsed 2000 comments...

parsed 4000 comments...

parsed 6000 comments...

...

parsed 18000 comments...

parsed 0 comments...

parsed 2000 comments...

parsed 4000 comments...

done!

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
print("Adding BOW")

vectCount = TfidfVectorizer(min_df=2, ngram_range=(1, 2))
vectCount.fit(X_train)

X_train_count = vectCount.transform(X_train)
# print(X_train_count.shape)
X_validate_count = vectCount.transform(X_validate)
import numpy as np
# X_count_train_np = np.asarray(X_train_count)
# X_count_validate_np = np.asarray(X_validate_count)
X_d2v_train_np = np.asarray(X_train_d2v)
X_d2v_validate_np = np.asarray(X_validate_d2v)

print(X_train_count.shape)
print(X_validate_count.shape)
print(X_d2v_train_np.shape)
print(X_d2v_validate_np.shape)

import scipy
X_merge_train = scipy.sparse.hstack((X_train_count, X_d2v_train_np))
X_merge_validate = scipy.sparse.hstack((X_validate_count, X_d2v_validate_np))

print(X_merge_train.shape)
print(X_merge_validate.shape)


Adding BOW

(20000, 363024)

(5000, 363024)

(20000, 400)

(5000, 400)

(20000, 363424)

(5000, 363424)

In [10]:
from sklearn.model_selection import GridSearchCV
# model = MLPClassifier(hidden_layer_sizes=(30, 30))
model = LogisticRegression(C=100)  # C=0.01
# param_grid = {'C': [.0095, 0.01, 0.015]}

# param_grid = {'C': [100, 150, 1000]}
# grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
# grid.fit(vectorizedComments, Y_words)

# print("Best cross-validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)
# print("Best estimator: ", grid.best_estimator_)
# param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
# print("Performing k-fold cross validation for our model...")
# kfoldscores = cross_val_score(logRegModel, X, Y, cv=5)
# # print(scores)
# print("Mean model accuracy = {}".format(kfoldscores.mean()))
print("training classifier model...")
model.fit(X_merge_train, Y_train)
# predictions = model.predict(X_merge_train)
predictions = model.predict(X_merge_validate)
accuracy = metrics.accuracy_score(Y_validate, predictions)
print("Model accuracy = {}".format(accuracy))


training classifier model...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Model accuracy = 0.8946

In [7]:
# print("Vectorizing test set")
# vectorizedComments_Z = []
# for comment in Z_words:
#     vectorizedComments_Z.append(model.infer_vector(comment))


In [8]:
# import time
# print("Running Model on new data")
# predictions = model.predict(Z_d2v)
# 
# f = open("predictions_" + str(int(time.time())) + ".csv", "w")
# f.write("Id,Category\n")
# i = 0
# for prediction in predictions:
#     f.write(str(i) + "," + str(int(prediction)))
#     f.write("\n")
#     i += 1
# f.close()
